<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#methods" data-toc-modified-id="methods-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>methods</a></span></li><li><span><a href="#the-'ghost-tensor'-issue" data-toc-modified-id="the-'ghost-tensor'-issue-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>the 'ghost tensor' issue</a></span></li><li><span><a href="#exorcism" data-toc-modified-id="exorcism-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>exorcism</a></span></li></ul></li><li><span><a href="#ethereal-evaded" data-toc-modified-id="ethereal-evaded-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ethereal evaded</a></span><ul class="toc-item"><li><span><a href="#pickle-testing" data-toc-modified-id="pickle-testing-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>pickle testing</a></span></li></ul></li><li><span><a href="#keras-example's-processes:" data-toc-modified-id="keras-example's-processes:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>keras example's processes:</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#create-base-network-w/-input-shape" data-toc-modified-id="create-base-network-w/-input-shape-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>create base network w/ input shape</a></span></li></ul></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Testing</a></span></li></ul></li><li><span><a href="#dio_3-results" data-toc-modified-id="dio_3-results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>dio_3 results</a></span></li><li><span><a href="#Transfer-Learning-(reappropriating-pretrained-models)" data-toc-modified-id="Transfer-Learning-(reappropriating-pretrained-models)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Transfer Learning (reappropriating pretrained models)</a></span><ul class="toc-item"><li><span><a href="#transfer-learning-can-be-done-later" data-toc-modified-id="transfer-learning-can-be-done-later-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>transfer learning can be done later</a></span><ul class="toc-item"><li><span><a href="#i-need-to-go-through-the-architecture-of-the-vgg-facenet-import;-something-in-it-isn't-compatible-with-my-y_train-array-shape" data-toc-modified-id="i-need-to-go-through-the-architecture-of-the-vgg-facenet-import;-something-in-it-isn't-compatible-with-my-y_train-array-shape-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>i need to go through the architecture of the vgg facenet import; something in it isn't compatible with my y_train array shape</a></span></li></ul></li><li><span><a href="#make-some-predictions" data-toc-modified-id="make-some-predictions-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>make some predictions</a></span></li></ul></li><li><span><a href="#Graphing" data-toc-modified-id="Graphing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Graphing</a></span></li></ul></div>

## imports

In [1]:
import keras
import numpy as np
import datetime

from keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict
from glob import glob
from random import choice, sample

# Florencia's imports
import tensorflow
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D,Activation, Dropout, BatchNormalization,  GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Subtract,Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import pandas as pd
import random
import cv2
import os
import pickle
from os import listdir
from tqdm import tqdm
from tensorflow.keras.applications.resnet50 import preprocess_input
from random import sample, choice
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models # inconsistency between 'from tensorflow.keras'
from tensorflow.keras import optimizers # and 'from keras'
from tensorflow.keras import backend as K # wonder if that's a bad thing
from keras.preprocessing.image import ImageDataGenerator # it probably is
from keras.backend import clear_session
from random import randint
from random import uniform as rfloat
import h5py
from tensorflow.keras.models import model_from_json # if you don't specift tensorflow. , it crashes
from keras import regularizers


Using TensorFlow backend.


## methods

In [35]:

def save_model(model, name): # format: 'name_optimizer_lr_epochs_batchsize'
    model_json = model.to_json()
    with open(name+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name+'.h5')
    print("Saved model to disk")
def load_model(name): # returns model obj with loaded weights
    json_file = open(name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name+".h5")
    print("Loaded model from disk")
    return loaded_model

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)



def pickle_obj(obj, filename): # designed with list of strings in mind
    urls = open(filename,"w+")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def unpickle_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def random_aug(img, img_gen):
    transform_parameters = { # full transform params list @ keras image preprocessing documentation
         'shear':randint(-8,8),
        'zx':rfloat(0.9,1.0), "zy":rfloat(0.9,1.0), 
        'theta':randint(-20,20), "flip_horizontal":bool(random.getrandbits(1))}
#     img_gen = ImageDataGenerator()
    return img_gen.apply_transform(img, transform_parameters)
def euclidean_distance(vects): # Euc_dist between 2 vectors
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes): # ?
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img)
def paths_to_arrays(paths_list):
    out = []
    for path in paths_list:
        out.append(read_img(path))
    return out

#  /Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/UTKface/test/
def get_jpg_lists(): # hardcoded ;(
    dir_path = '/Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/'
    train_part = 'UTKface/train/'
    val_part = 'UTKface/validation/'
    test_part = 'UTKface/test/'
    # no overlap between train, val, test imgs
    train_path = dir_path+train_part # 2008
    val_path = dir_path+val_part # 95
    test_path = dir_path+test_part # 155, distinct from val
    
    trains = glob(train_path + '*.jpg')
    tests = glob(test_path + '*.jpg')
    vals = glob(val_path + '*.jpg')
    return trains, tests, vals

def jpgs_to_arrays(trains, tests, vals):
    train_data = paths_to_arrays(trains)
    test_data = paths_to_arrays(tests)
    val_data = paths_to_arrays(vals)
    return train_data, test_data, val_data


from copy import deepcopy

def remove_array(L,arr): # deletes array from list
    ind = 0 # list.remove(e) & del list[ind] both use ==
    size = len(L) # that doesn't play nice with arrays, so we need a custom remove_ar_from_list()
    while ind != size and not np.array_equal(L[ind],arr): # courtesy of stackoverflow
        ind += 1
    if ind != size:
        L.pop(ind)
    else:
        raise ValueError('array not found in list.') 

def get_pairs(imgs_list): # creates 50/50 split of 'same' and 'dif' image pairs & labels. len=n*5
    pairs = [] # x
    labels = [] # y
    print(f'making pairs from {len(imgs_list)} images...')
    for c, img in enumerate(imgs_list):
        temp = imgs_list # shallow copy messed this up
#         remove_array(temp, img) # make list without current array
        for i in range(0,4): # make 4 'same' and 'dif' pairs for each base_image
            # same pair
            pairs.append([img, img])
            labels.append(1) # true
            # dif pair
            print(f'getting rand_ind between 0 and {len(temp)}')
            indx = randint(0,len(temp)-1)
            while indx==c: # if rand is same 
                indx = randint(0,len(temp)-1) # avoid deepcopy memory time
            print(f'index: {indx}')
            pairs.append([img, temp[indx]])
            labels.append(0)
            print(f'total {len(pairs)}')
        print(f'done with image #{c} \n')
    return pairs, labels # returns lists for x, y. pairs+labels matched up by index


def xy_split(train_list, test_list, val_list):
    x_train, y_train = get_pairs(train_data)
    x_test, y_test = get_pairs(test_data)
    x_val, y_val = get_pairs(val_data)
    return x_train,y_train, x_test,y_test, x_val,y_val


# ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
def generator(all_pairs, labels, aug_gen_obj, batch_size=16):
    '''is there a reason for a specific batch_size? not really. tradeoffs in speed vs. efficiency
    i'm sampling randomly from all pairs, so the data is shuffled'''
    while True: # this makes the whole thing work, dunno why
        batch_faces = sample(all_pairs, batch_size) # get a batch of face pairs [face1, face2] w/o replacement
        faces_left, faces_right = [], [] # outputs. holds pairs
        labels = np.ones(batch_size) # label vector output. length matches number of pairs in batch
        for c, face_pair in enumerate(batch_faces): # [left_face, right_face]
            if np.array_equal(face_pair[0], face_pair[1]): # it's the same person
                face_pair[1] = random_aug(face_pair[1], aug_gen_obj) # randomly augment the right_face
                labels[c] = 1 # set corresponding label to True
            else: # it's a different person
                labels[c] = 0
            faces_left.append(face_pair[0])
            faces_right.append(face_pair[1])
        yield [faces_left, faces_right], labels

## the 'ghost tensor' issue
- in the earlier prototypes, i encountered a perplexingly pernicious phantom 
    - data process:
        - glob jpgs from train,test,val folders
        - 3 lst of jpg strings --> 3 lst of jpg arrays
        - lst_jpg_ars --> n_pairs of jpgs, stored as list
            - converting list to array is shape=(num_pairs, 2, 200, 200, 3)
        - feed this list as train
    - shape of each folder w/ n=num_pics should be (n,200,200,3) for dimensions+color
        - for train and val, this was so
        - but shape(test) = (n,)
            - and shape(test[0]) = (200,200,3)
                - spooky
            - this prevents me from loading test images into model.predict()
                - _oh no_

    - investigating supernatural phenomena:
        - test folder has 156 images
        - glob was picking up 155
            - there's a hidden file
            - a ghost
        - or one of the jpgs is misnamed and not being picked up
        - but how would that cause the shape to be misread?


## exorcism
- since we know that the train_jpg_list gives the right output
     - we'll just sample a new test batch from the train jpg list after glob from folder
     - this way, we don't go near the haunted folder --> our data is not possessed

In [36]:
def avoid_ghosts(train_jpgs, n=160): # 2008-160=1848. both fit batch_size=16
    tests = random.sample(train_jpgs, n)
    for jpg in tests:
        if jpg in train_jpgs:
            train_jpgs.remove(jpg)
    return tests

In [4]:
train_data, test_data, val_data = get_jpg_lists()

In [6]:
# shun spirits
test_data = avoid_ghosts(train_data)

In [7]:
# convert jpgs to arrays
train_data, test_data, val_data = jpgs_to_arrays(train_data, test_data, val_data)

In [8]:
print(np.shape(train_data))
print(np.shape(test_data))
print(np.shape(val_data))

(1848, 200, 200, 3)
(160, 200, 200, 3)
(95, 200, 200, 3)


In [9]:
# make pairs from lists of arrays
x_train,y_train, x_test,y_test, x_val,y_val = xy_split(train_data, test_data, val_data)

making pairs from 1848 images...
getting rand_ind between 0 and 1848
index: 908
total 2
getting rand_ind between 0 and 1848
index: 773
total 4
getting rand_ind between 0 and 1848
index: 449
total 6
getting rand_ind between 0 and 1848
index: 1621
total 8
done with image #0 

getting rand_ind between 0 and 1848
index: 1571
total 10
getting rand_ind between 0 and 1848
index: 27
total 12
getting rand_ind between 0 and 1848
index: 929
total 14
getting rand_ind between 0 and 1848
index: 649
total 16
done with image #1 

getting rand_ind between 0 and 1848
index: 1430
total 18
getting rand_ind between 0 and 1848
index: 408
total 20
getting rand_ind between 0 and 1848
index: 1646
total 22
getting rand_ind between 0 and 1848
index: 261
total 24
done with image #2 

getting rand_ind between 0 and 1848
index: 1745
total 26
getting rand_ind between 0 and 1848
index: 1833
total 28
getting rand_ind between 0 and 1848
index: 1107
total 30
getting rand_ind between 0 and 1848
index: 876
total 32
done w

total 2204
getting rand_ind between 0 and 1848
index: 1248
total 2206
getting rand_ind between 0 and 1848
index: 367
total 2208
done with image #275 

getting rand_ind between 0 and 1848
index: 524
total 2210
getting rand_ind between 0 and 1848
index: 466
total 2212
getting rand_ind between 0 and 1848
index: 634
total 2214
getting rand_ind between 0 and 1848
index: 259
total 2216
done with image #276 

getting rand_ind between 0 and 1848
index: 1452
total 2218
getting rand_ind between 0 and 1848
index: 437
total 2220
getting rand_ind between 0 and 1848
index: 1113
total 2222
getting rand_ind between 0 and 1848
index: 239
total 2224
done with image #277 

getting rand_ind between 0 and 1848
index: 47
total 2226
getting rand_ind between 0 and 1848
index: 228
total 2228
getting rand_ind between 0 and 1848
index: 1253
total 2230
getting rand_ind between 0 and 1848
index: 596
total 2232
done with image #278 

getting rand_ind between 0 and 1848
index: 1591
total 2234
getting rand_ind betwee

getting rand_ind between 0 and 1848
index: 944
total 4576
done with image #571 

getting rand_ind between 0 and 1848
index: 382
total 4578
getting rand_ind between 0 and 1848
index: 635
total 4580
getting rand_ind between 0 and 1848
index: 1843
total 4582
getting rand_ind between 0 and 1848
index: 635
total 4584
done with image #572 

getting rand_ind between 0 and 1848
index: 1049
total 4586
getting rand_ind between 0 and 1848
index: 304
total 4588
getting rand_ind between 0 and 1848
index: 1363
total 4590
getting rand_ind between 0 and 1848
index: 849
total 4592
done with image #573 

getting rand_ind between 0 and 1848
index: 1536
total 4594
getting rand_ind between 0 and 1848
index: 585
total 4596
getting rand_ind between 0 and 1848
index: 1523
total 4598
getting rand_ind between 0 and 1848
index: 665
total 4600
done with image #574 

getting rand_ind between 0 and 1848
index: 1564
total 4602
getting rand_ind between 0 and 1848
index: 917
total 4604
getting rand_ind between 0 and 1

getting rand_ind between 0 and 1848
index: 1508
total 6230
getting rand_ind between 0 and 1848
index: 1681
total 6232
done with image #778 

getting rand_ind between 0 and 1848
index: 1062
total 6234
getting rand_ind between 0 and 1848
index: 778
total 6236
getting rand_ind between 0 and 1848
index: 489
total 6238
getting rand_ind between 0 and 1848
index: 615
total 6240
done with image #779 

getting rand_ind between 0 and 1848
index: 1400
total 6242
getting rand_ind between 0 and 1848
index: 874
total 6244
getting rand_ind between 0 and 1848
index: 636
total 6246
getting rand_ind between 0 and 1848
index: 1549
total 6248
done with image #780 

getting rand_ind between 0 and 1848
index: 110
total 6250
getting rand_ind between 0 and 1848
index: 1009
total 6252
getting rand_ind between 0 and 1848
index: 653
total 6254
getting rand_ind between 0 and 1848
index: 796
total 6256
done with image #781 

getting rand_ind between 0 and 1848
index: 350
total 6258
getting rand_ind between 0 and 1

total 8532
getting rand_ind between 0 and 1848
index: 959
total 8534
getting rand_ind between 0 and 1848
index: 1055
total 8536
done with image #1066 

getting rand_ind between 0 and 1848
index: 1783
total 8538
getting rand_ind between 0 and 1848
index: 586
total 8540
getting rand_ind between 0 and 1848
index: 923
total 8542
getting rand_ind between 0 and 1848
index: 58
total 8544
done with image #1067 

getting rand_ind between 0 and 1848
index: 533
total 8546
getting rand_ind between 0 and 1848
index: 947
total 8548
getting rand_ind between 0 and 1848
index: 128
total 8550
getting rand_ind between 0 and 1848
index: 723
total 8552
done with image #1068 

getting rand_ind between 0 and 1848
index: 159
total 8554
getting rand_ind between 0 and 1848
index: 614
total 8556
getting rand_ind between 0 and 1848
index: 293
total 8558
getting rand_ind between 0 and 1848
index: 803
total 8560
done with image #1069 

getting rand_ind between 0 and 1848
index: 1074
total 8562
getting rand_ind betw

getting rand_ind between 0 and 1848
index: 1680
total 10908
getting rand_ind between 0 and 1848
index: 910
total 10910
getting rand_ind between 0 and 1848
index: 334
total 10912
done with image #1363 

getting rand_ind between 0 and 1848
index: 591
total 10914
getting rand_ind between 0 and 1848
index: 1475
total 10916
getting rand_ind between 0 and 1848
index: 1683
total 10918
getting rand_ind between 0 and 1848
index: 794
total 10920
done with image #1364 

getting rand_ind between 0 and 1848
index: 94
total 10922
getting rand_ind between 0 and 1848
index: 484
total 10924
getting rand_ind between 0 and 1848
index: 15
total 10926
getting rand_ind between 0 and 1848
index: 536
total 10928
done with image #1365 

getting rand_ind between 0 and 1848
index: 404
total 10930
getting rand_ind between 0 and 1848
index: 110
total 10932
getting rand_ind between 0 and 1848
index: 1748
total 10934
getting rand_ind between 0 and 1848
index: 1429
total 10936
done with image #1366 

getting rand_ind

getting rand_ind between 0 and 1848
index: 1343
total 13118
getting rand_ind between 0 and 1848
index: 776
total 13120
done with image #1639 

getting rand_ind between 0 and 1848
index: 1284
total 13122
getting rand_ind between 0 and 1848
index: 1797
total 13124
getting rand_ind between 0 and 1848
index: 1527
total 13126
getting rand_ind between 0 and 1848
index: 1109
total 13128
done with image #1640 

getting rand_ind between 0 and 1848
index: 228
total 13130
getting rand_ind between 0 and 1848
index: 1141
total 13132
getting rand_ind between 0 and 1848
index: 1741
total 13134
getting rand_ind between 0 and 1848
index: 1168
total 13136
done with image #1641 

getting rand_ind between 0 and 1848
index: 853
total 13138
getting rand_ind between 0 and 1848
index: 1085
total 13140
getting rand_ind between 0 and 1848
index: 577
total 13142
getting rand_ind between 0 and 1848
index: 1211
total 13144
done with image #1642 

getting rand_ind between 0 and 1848
index: 1131
total 13146
getting 

total 624
done with image #77 

getting rand_ind between 0 and 160
index: 22
total 626
getting rand_ind between 0 and 160
index: 142
total 628
getting rand_ind between 0 and 160
index: 49
total 630
getting rand_ind between 0 and 160
index: 156
total 632
done with image #78 

getting rand_ind between 0 and 160
index: 151
total 634
getting rand_ind between 0 and 160
index: 149
total 636
getting rand_ind between 0 and 160
index: 109
total 638
getting rand_ind between 0 and 160
index: 6
total 640
done with image #79 

getting rand_ind between 0 and 160
index: 144
total 642
getting rand_ind between 0 and 160
index: 103
total 644
getting rand_ind between 0 and 160
index: 32
total 646
getting rand_ind between 0 and 160
index: 40
total 648
done with image #80 

getting rand_ind between 0 and 160
index: 60
total 650
getting rand_ind between 0 and 160
index: 17
total 652
getting rand_ind between 0 and 160
index: 116
total 654
getting rand_ind between 0 and 160
index: 98
total 656
done with image

In [10]:
print(np.shape(x_train))
print(np.shape(x_test))
print(np.shape(x_val))

(14784, 2, 200, 200, 3)
(1280, 2, 200, 200, 3)
(760, 2, 200, 200, 3)


# ethereal evaded

## pickle testing


In [11]:
# pickle_obj(x_train, 'x_train')
# pickle_obj(y_train, 'y_train')
# pickle_obj(x_test, 'x_test')
# pickle_obj(y_test, 'y_test')
# pickle_obj(x_val, 'x_val')
# pickle_obj(y_val, 'y_val')

In [12]:
# x_train = unpickle_obj('x_train')
# y_train = unpickle_obj('y_train')
# x_test = unpickle_obj('x_test')
# y_test = unpickle_obj('y_test')
# x_val = unpickle_obj('x_val')
# y_val = unpickle_obj('y_val')

# keras example's processes:

### create base network w/ input shape

In [38]:
K.clear_session() # wipe GPU memory

- ideas from karolina:
    - initializer on first or all layers
    - regularizer on last or all layers

In [39]:
def create_base_network(input_shape): 
    inp = Input(shape=input_shape) # consider size of filters
    x = Conv2D(64,(10,10), activation='relu',input_shape=input_shape)
    x = MaxPooling2D()
    x = Conv2D(128,(7,7),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D()
    x = Conv2D(128,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D()
    x = Conv2D(128,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D() # 512 kills runtime
    x = Conv2D(256,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = Flatten()(inp)
    x = Dense(4096, activation='relu',
             kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
    return Model(inp, x) 
input_shape = np.shape(x_train[0][0])
base_network = create_base_network(input_shape)

In [40]:
model = 'a'

In [41]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a) # run left/right inputshapes through network
processed_b = base_network(input_b)

# distance = Lambda(euclidean_distance,
#          output_shape=eucl_dist_output_shape),
#             [processed_a, processed_b])
distance = Lambda(euclidean_distance, # euclid distance to start. contrastive later?
                      output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
model = Model([input_a, input_b], distance) # keras API model for input, output

In [42]:
img_gen = ImageDataGenerator()

from keras.callbacks import EarlyStopping, ModelCheckpoint
patience_=3 # play around with this
es = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=patience_,verbose=1,restore_best_weights=True)
calb = []
calb.append(es) # early stopping to callbacks

- notes:
    - 4 conv, flatten, 1 Dense
        - Sigmoid on Dense gave .14acc early stop
        - switching back to Relu is looking better
        - maybe try 4096
            - oof, 4096 was ambitious. so was 2048. my laptop is hot.
            - 1024 is still a hefty increase in runtime (10s-->2min / epoch), but might be worth it
            - if increasing nodes in final layer helps, maybe bump to 2048 run overnight
    - 1024 wasn't quite what i wanted
        -  peaked at 72,75 after 12 epochs / 45
            - is patience=5 too low? maybe 6
        - try again patience=6, changing the last relu to a sigmoid

In [25]:
# rms = optimizers.RMSprop()
from tensorflow.keras.optimizers import Adam

model.compile(loss=contrastive_loss, 
              optimizer = optimizers.Adam(0.00001), metrics=['acc'])
history = model.fit_generator(generator(x_train,y_train,img_gen),
                validation_data = generator(x_val,y_val,img_gen),
                epochs=50, verbose=1, steps_per_epoch=100, 
                validation_steps=10, callbacks=calb)

Epoch 1/10
100/100 [==============================] - 222s 2s/step - loss: 52771.7194 - acc: 0.1369 - val_loss: 53.2314 - val_acc: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 222s 2s/step - loss: 2001.2251 - acc: 0.3413 - val_loss: 43.1312 - val_acc: 0.7312
Epoch 3/10
100/100 [==============================] - 219s 2s/step - loss: 1302.7550 - acc: 0.5769 - val_loss: 47.9546 - val_acc: 0.6750
Epoch 4/10
100/100 [==============================] - 195s 2s/step - loss: 1622.9914 - acc: 0.6737 - val_loss: 43.7274 - val_acc: 0.7250
Epoch 5/10
100/100 [==============================] - 193s 2s/step - loss: 947.4621 - acc: 0.6775 - val_loss: 44.0571 - val_acc: 0.8188
Epoch 6/10
100/100 [==============================] - 193s 2s/step - loss: 1252.3655 - acc: 0.6862 - val_loss: 43.0807 - val_acc: 0.8250
Epoch 7/10
100/100 [==============================] - 211s 2s/step - loss: 10102.6604 - acc: 0.4975 - val_loss: 65.3353 - val_acc: 0.2750
Epoch 8/10
100/100 [================

In [26]:
import h5py
from tensorflow.keras.models import model_from_json # if you don't specift tensorflow. , it crashes
def save_model(model, name): # format: 'name__optimizer_lr_epochs_batchsize'
    model_json = model.to_json()
    with open(name+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name+'.h5')
    print("Saved model to disk")
def load_model(name): # returns model obj with loaded weights
    json_file = open(name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name+".h5")
    print("Loaded model from disk")
    return loaded_model

In [27]:
name = 'bruno__1__66testacc_conv_dense2048relu'
save_model(model, name)

Saved model to disk


## Testing

In [28]:
x_tr = np.array(x_train)
y_tr = np.array(y_train)
x_te = np.array(x_test)
y_te = np.array(y_test)

In [ ]:
# pickle_obj(x_tr, 'x_tr')
# pickle_obj(y_train, 'y_tr')
# pickle_obj(x_te, 'x_te')
# pickle_obj(y_te, 'y_te')

# pickle max size 4gb?

In [29]:
y_pred_train = model.predict([x_tr[:, 0], x_tr[:, 1]])

In [30]:
tr_acc = compute_accuracy(y_train, y_pred_train)
print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))

* Accuracy on training set: 55.00%


In [31]:
y_pred_test = model.predict([x_te[:, 0], x_te[:, 1]])

In [32]:
te_acc = compute_accuracy(y_test, y_pred_test)
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on test set: 62.97%


# dio_3 results
- decent score of 63 on the test images. not bad for a 6-layer network, 2000 images & 30min runtime.
    - 55% on train, though. odd.
 

In [ ]:
np.shape(x_te)

In [ ]:
type(x_te)

In [ ]:
np.shape(x_te[:, 0]) # you pass in [list_left_tensors, list_right_tensors]

# Transfer Learning (reappropriating pretrained models)

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

base_model = VGG19(weights='imagenet', include_top=False)
base_model.summary()

In [ ]:
def create_base_network(input_shape): 
    inp = Input(shape=input_shape)
    x = Conv2D(64,(10,10), activation='relu',input_shape=input_shape)
    x = MaxPooling2D()
    x = Conv2D(128,(7,7),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D()
    x = Conv2D(128,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    x = MaxPooling2D()
    x = Conv2D(256,(4,4),activation='relu',
              kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))
    # dense
    x = Flatten()(inp)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01),
#                 activity_regularizer=regularizers.l1(0.01))(x)
#     x = Dropout(0.1)(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.25)(x)
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.15)(x)
    x = Dense(1024, activation='relu', # don't softmax 
             kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x) # should squish it to 1?
    print(type(x))
    return Model(inp, x) # this is the magic. Keras' API handles the low-level shape matching for you 
input_shape = np.shape(x_train[0][0])
base_network = create_base_network(input_shape)

In [ ]:
input_shape

In [ ]:
print(type(vgg))

In [ ]:
inp = Input(shape=input_shape)
# vgg_s = Model(inp, vgg)

In [ ]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = vgg(input_a)
processed_b = vgg(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance) # keras API model for input, output

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_gen = ImageDataGenerator()

from keras.callbacks import EarlyStopping, ModelCheckpoint
patience_=6
es = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=patience_,verbose=1,restore_best_weights=True)
calb = []
calb.append(es)

In [ ]:
# rms = optimizers.RMSprop()
from tensorflow.keras.optimizers import Adam

model.compile(loss=contrastive_loss, optimizer = optimizers.Adam(0.00001), metrics=['acc'])
history = model.fit_generator(generator(x_train,y_train,img_gen),
                   validation_data = generator(x_val,y_val,img_gen),
                    epochs=45, verbose=1, steps_per_epoch=100, validation_steps=10,
                    callbacks=calb
                   )

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19


# Define Model Architecture 
def siamese_model(pretrained_model=None):
    #left and right inputs for siamese network
    left_image = layers.Input(shape = (200, 200, 3))
    right_image = layers.Input(shape = (200, 200, 3))
    
    #start model
    model = models.Sequential()
    #check if there is a pretrained model
    if pretrained_model is not None:
        model.add(pretrained_model)
        
    model.add(layers.Dense(128, activation = "relu"))
    pretrained_model.trainable = False
    
    x1 = model(left_image) # base_net(left_2)
    x2 = model(right_image)
    
    L2_normalized_layer_1 = layers.Lambda(lambda x: K.l2_normalize(x, axis = 1))
    X1_normal = L2_normalized_layer_1(x1)
    X2_normal = L2_normalized_layer_1(x2)

    distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

    model = Model([input_a, input_b], distance)
    
#     L1_layer = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
#     L1_distance = L1_layer([X1_normal, X2_normal])
    
#     prediction = layers.Dense(1, activation = "sigmoid")(L1_distance)
    
#     siamese_net = models.Model(inputs = [left_image, right_image], outputs = prediction)

    model.compile(loss = contrastive_loss, metrics = ["acc"], optimizer = optimizers.Adam(0.00001))
    
    return model

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False)
vgg_model = siamese_model(pretrained_model=base_model)

In [ ]:
# kinship_model.fit_generator(gen(train, train_person_to_images_map, batch_size = 7),
#                     validation_data =       gen(val, val_person_to_images_map, batch_size = 7), 
#                     epochs = 5, verbose = 2, 
#                     steps_per_epoch = 100, 
#                     validation_steps = 10)

In [ ]:
# history = vgg_model.fit_generator(generator(x_train,y_train,img_gen),
#                    validation_data = generator(x_val,y_val,img_gen),
#                     epochs=45, verbose=1, steps_per_epoch=100, validation_steps=10,
#                     callbacks=calb
#                    )

## transfer learning can be done later
### i need to go through the architecture of the vgg facenet import; something in it isn't compatible with my y_train array shape
and it's not the val_data

- a better option would be to start from scratch using Facebook's Facenet implementation. it's popular enough that people have written keras wrappers for it, so i can just go from there

## make some predictions
"ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (200, 200, 3)"
- so model.predict(x) needs x to be (2, 200, 200, 3). an array that contains a pair of 3d image-tensors.

In [33]:
y_pred_test = model.predict([x_te[:, 0], x_te[:, 1]])

In [34]:

print(type(x_te),np.shape(x_te))

<class 'numpy.ndarray'> (1280, 2, 200, 200, 3)


- ah, so we have to put out swipe_user_predictions in as a 5d array
- let's go get the survey results 

- get list of user_images
    - resize, convert to array
- for each user_image:
    - get list of swipes: ['Option 1', 'Option 5', 'Option 7']
        - we're gonna have to match these to the actual jpgs
            - didn't i just have them swipe on Test?
                - the original 155 tests (the haunted folder)
            - yes. yes i did. and i ordered the survey matching Test folder's order(name)
                    - this does bring a smile to my face
            - so if i just remove the 5 data_student faces from Test, i can assign option 1 to the first index, etc
                - use a dict
                

In [ ]:
# options = []
# for i in range(1,151):
#     options.append('Option '+str(i))
# # options 

In [ ]:
# # remember, this is the cursed folder
# # we'll just hardcode it for now.
# path = '/Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/UTKface/test/'

In [ ]:
left = x_test[1][0]
# plt.imshow(left)

In [ ]:
right = x_test[1][1]
# plt.imshow(right)

In [ ]:
predictions_raw = model.predict(vstack_batch_input, batch_size=len(batch_input))
predictions = np.argmax(predictions_raw,axis=1)
tot_matches = np.sum(predictions)
if tot_matches == 1:
    # ... good times, whatever face[N] has prediction[N] == 1 is the matching face

In [ ]:
dif_pair = np.array([left, right])
np.shape(dif_pair)

In [ ]:
type(dif_pair)

In [ ]:
model.predict(dif_pair)

# Graphing

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline 

# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# epochs = range(len(acc))
# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()
# plt.figure()
# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()
# plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_te = np.array(y_test)

In [ ]:
print(type(y_te),type(y_pred))

In [ ]:
y_pred.max()

In [ ]:
score = model.evaluate(x_te[:, 0], x_te[:, 1])

In [ ]:
test_loss, test_acc = model.evaluate_generator(generator, steps=50)
print('test acc:', test_acc)

In [ ]:
p = accuracy_score(y_te, y_pred_test)

- visualization checklist:
    - what the net 'sees' at each layer
    - AOC over epochs, everyone does this. definitely need it
    - confusion matrix